In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pycaret

     |████████████████████████████████| 256kB 12.3MB/s 
     |████████████████████████████████| 65.8MB 56kB/s 
     |████████████████████████████████| 1.8MB 45.6MB/s 
     |████████████████████████████████| 174kB 60.2MB/s 
     |████████████████████████████████| 245kB 57.7MB/s 
     |████████████████████████████████| 276kB 60.0MB/s 
     |████████████████████████████████| 14.2MB 185kB/s 
     |████████████████████████████████| 6.8MB 47.7MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 1.6MB 38.9MB/s 
     |████████████████████████████████| 157.5MB 26kB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 604kB 54.3MB/s 
     |████████████████████████████████| 3.1MB 44.6MB/s 
     |████████████████████████████████| 163kB 60.4MB/s 
     |████████████████████████████████| 153kB 63.3MB/s 
     |████████████████████████████████| 348kB 59.0MB/s 
     |████████████████████████████████| 81kB 9.1MB/

In [3]:
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
from pycaret.classification import *

In [4]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

In [5]:
path = '/content/drive/Shareddrives/dacon/404/preprocessed/mk2_with_categorized/'

test_err_42_counts = pd.read_csv(path + '테스트_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])
train_err_42_counts = pd.read_csv(path + '트레인_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])

test_err_code_frq = pd.read_csv(path + '테스트_에러_에러코드별_발생횟수.csv', index_col=['Unnamed: 0'])
train_err_code_frq = pd.read_csv(path +'트레인_에러_에러코드별_발생횟수.csv', index_col=['Unnamed: 0'])

test_err_sum_byday = pd.read_csv(path +'테스트_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])
train_err_sum_byday = pd.read_csv(path +'트레인_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])

test_qui_frq = pd.read_csv(path +'테스트_퀼리티_항목별_많이등장.csv', index_col=['Unnamed: 0'])
train_qui_frq = pd.read_csv(path +'트레인_퀼리티_항목별_많이등장.csv', index_col=['Unnamed: 0'])

test_qui_sum_byday = pd.read_csv(path +'테스트_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])
train_qui_sum_byday = pd.read_csv(path +'트레인_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])

In [6]:
# 각 사용자가 42가지 에러타입을 몇 번씩 겪었는지 나타냄
train_err_42_counts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


In [7]:
# 각 사용자가 가장 자주 겪은 에러코드를 나타냄(위에는 에러타입, 여기는 에러코드)
train_err_code_frq

,Most_Err_Code,Second_Err_Code
10000,1,0
10001,1,standby
10002,1,0
10003,1,0
10004,1,connection timeout
...,...,...
24995,1,0
24996,1,ETC
24997,1,0
24998,1,0


In [8]:
# 각 사용자가 퀼리티 1 부터 퀼리티 12까지 자주 겪은 퀼리티 로그를 나타냄
# _1은 가장 자주 겪은 로그이며, _2는 두 번째로 자주 겪은 로그임. (None은 2번째 로그가 없음, 한 가지 오류만 겪음)
train_qui_frq

################ 이거는 카테고리 기준 등장빈도이며... 이를 확률로 치환하는 새로운 방법을 시도해본다.

,q1_1,q1_2,q2_1,q2_2,q5_1,q5_2,q6_1,q6_2,q7_1,q7_2,q8_1,q8_2,q9_1,q9_2,q10_1,q10_2,q11_1,q11_2,q12_1,q12_2
10000,0.0,None,0.0,None,0.0,4.0,0.0,None,0.0,None,0.0,None,0.0,None,7.0,4.0,0.0,None,0.0,None
10002,0.0,-1.0,0.0,-1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,None,0.0,1.0,3.0,5.0,0.0,-1.0,0.0,None
10004,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,None,0.0,None,1.0,3.0,0.0,-1.0,0.0,None
10005,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,None,0.0,None,4.0,5.0,0.0,-1.0,0.0,None
10006,0.0,None,0.0,None,0.0,4.0,0.0,1.0,0.0,1.0,0.0,None,0.0,None,4.0,None,0.0,None,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,1.0,None,0.0,-1.0,0.0,None
24992,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,5.0,3.0,0.0,-1.0,0.0,None
24993,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,5.0,3.0,0.0,-1.0,0.0,None
24995,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,3.0,2.0,0.0,-1.0,0.0,None


In [10]:
# 일단 자료들 날짜형식이나 콤마(,) 제거 등 간단한 전처리 진행

train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

In [11]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [12]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data

###### 그냥 쓰자니 불일치가 너무 많아서... 카테고리화 해서 확률로 전환한다.

train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

In [13]:
prob_list = list(set(train_prob.user_id))

train_qui3['problems'] = train_qui3.user_id.isin(prob_list)
train_qui3_tr = train_qui3[train_qui3['problems']].dropna()
train_qui3_fls = train_qui3[train_qui3['problems'] != True].dropna()

In [14]:
def probability(trainset, testset):
  train = trainset.copy()
  test = testset.copy()

  cols = list(train.loc[:,'quality_1':'quality_12'].columns)

  train['problems'] = train.user_id.isin(prob_list)
  train_tr = train[train['problems']]
  train_fls = train[train['problems'] != True]


  for col in tqdm(cols):
    temp_ori = train[col].value_counts()
    temp_tr = train_tr[col].value_counts()
    temp_fls = train_fls[col].value_counts()

    ori = pd.DataFrame(temp_ori)
    tr = pd.DataFrame(temp_tr)

    probs = tr/ori
    for ind in list(probs.index):
      train[col][train[col] == ind] = probs.loc[ind,:].values[0]
      test[col][test[col] == ind] = probs.loc[ind,:].values[0]
  train.drop(['problems'], axis = 1, inplace = True)
  return train, test

In [15]:
train_qui_prob, test_qui_prob = probability(train_qui3, test_qui3)

In [16]:
# 일단 확률로 바꾼 값들을 같은 ID끼리 평균으로 묶는다.

train_qui_probs = train_qui_prob.groupby(by = 'user_id').mean()
test_qui_probs = test_qui_prob.groupby(by = 'user_id').mean()

In [17]:
#################### test_qui_frq 대신에 이걸 넣자. ########################

test_qui_probs

,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,,,,,,,,,,
30000,0.453202,0.470339,0.399646,0.456372,0.478145,0.463237,0.470470,0.390104,0.453996,0.468568
30001,0.471144,0.491062,0.449704,0.471127,0.465027,0.463237,0.470470,0.476673,0.471726,0.468568
30002,0.462173,0.480701,0.428165,0.462095,0.465027,0.463237,0.470470,0.490867,0.462861,0.468568
30003,0.453202,0.470339,0.399759,0.456372,0.472898,0.463237,0.470470,0.338923,0.453996,0.468568
30004,0.471144,0.491062,0.429772,0.474437,0.478145,0.463237,0.470470,0.366937,0.471726,0.468568
...,...,...,...,...,...,...,...,...,...,...
44990,0.461177,0.479549,0.429800,0.461092,0.465027,0.463237,0.470470,0.487934,0.461876,0.468568
44993,0.456193,0.475545,0.408818,0.459383,0.471586,0.463237,0.442244,0.402184,0.456951,0.468568
44994,0.518989,0.546323,0.520579,0.519299,0.465027,0.463237,0.470470,0.390104,0.519006,0.468568


In [18]:
# Error 데이터를 일일 단위로 쪼개서 각 사용자가 날짜별로 겪은 에러코드의 누계
# 0일은 10월 31일, 마지막은 11월 30일
train_err_sum_byday

,er_sum_0,er_sum_1,er_sum_2,er_sum_3,er_sum_4,er_sum_5,er_sum_6,er_sum_7,er_sum_8,er_sum_9,er_sum_10,er_sum_11,er_sum_12,er_sum_13,er_sum_14,er_sum_15,er_sum_16,er_sum_17,er_sum_18,er_sum_19,er_sum_20,er_sum_21,er_sum_22,er_sum_23,er_sum_24,er_sum_25,er_sum_26,er_sum_27,er_sum_28,er_sum_29,er_sum_30
10000,0,11,20,38,43,53,62,82,89,94,104,118,123,140,149,165,174,182,199,218,223,230,235,240,245,254,265,277,295,308,317
10001,0,11,61,90,138,180,209,222,246,299,351,374,413,421,462,486,501,531,556,583,628,653,671,689,716,727,785,2237,2275,2337,2365
10002,0,10,23,36,51,60,68,85,96,108,118,129,140,145,160,173,188,193,202,217,222,232,248,251,257,268,273,289,289,297,306
10003,0,9,23,33,38,54,59,61,72,77,96,120,126,132,134,136,151,157,164,179,184,191,212,223,240,246,254,275,286,300,306
10004,0,25,46,95,123,134,162,180,209,229,283,302,323,340,360,375,390,408,449,457,559,587,603,621,644,671,690,714,732,755,777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,41,58,76,76,100,103,133,155,172,194
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
24997,0,9,43,77,141,163,194,218,248,274,292,301,320,339,377,408,418,445,464,496,525,550,579,621,654,695,718,748,779,813,826
24998,0,7,7,17,17,24,24,32,32,39,76,83,83,89,91,96,102,102,111,112,118,120,125,131,131,137,137,144,144,151,155


In [19]:
# Quality 데이터를 일일 단위로 쪼개서 퀼리티 로그가 사용자 별로 몇 번 수집되었나
# 누적, 마지막 11월 30일 자료는 최종 몇 번의 로그 수집이 있었는가를 의미함.
train_qui_sum_byday

,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30
10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
10002,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0
10004,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
24992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
24993,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
24995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0


In [20]:
train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

train_err3 = train_err2[['user_id', 'model_nm', 'fwver']]
test_err3 = test_err2[['user_id', 'model_nm', 'fwver']]

train_err3 = train_err3.drop_duplicates(subset = ['user_id'])
test_err3 = test_err3.drop_duplicates(subset = ['user_id'])

train_err3.set_index('user_id', inplace = True)
test_err3.set_index('user_id', inplace = True)

In [21]:
# train_err_42_counts, train_err_code_frq, train_qui_frq, train_err_sum_byday, train_qui_sum_byday
# test_err3, test_err_42_counts, test_err_code_frq, test_qui_frq, test_err_sum_byday, test_qui_sum_byday
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [22]:
test_qui_frq = test_qui_frq.astype(str)
train_qui_frq = train_qui_frq.astype(str)

In [25]:
testset = pd.concat([test_err3, test_err_42_counts,  test_qui_sum_byday], axis = 1) #test_err_sum_byday,
trainset = pd.concat([train_err3, train_err_42_counts, train_qui_sum_byday], axis = 1) #train_err_sum_byday
testset = testset.fillna(0)
trainset = trainset.fillna(0)
trainset.iloc[:,2:] = scaler.fit_transform(trainset.iloc[:,2:])
testset.iloc[:,2:] = scaler.transform(testset.iloc[:,2:])

In [26]:
testset

,model_nm,fwver,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30
30000,model_1,04.16.3553,0.0,0.0,0.000000,0.000000,0.009342,0.000557,0.000372,0.0,0.0,0.0,0.002183,0.002182,0.000000,0.000218,0.043906,0.024183,0.019565,0.073171,0.0,0.043478,0.0,0.061653,0.049368,0.001745,0.025641,0.135021,0.000000,0.000000,0.0,0.0,0.002206,0.000355,0.087432,0.000000,0.001208,0.500000,0.500000,0.000000,0.002073,0.054277,0.000000,0.060606,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003049,0.002941,0.002841
30001,model_3,05.15.2138,0.0,0.0,0.000412,0.000502,0.000452,0.002227,0.001489,0.0,0.0,0.0,0.001637,0.002182,0.000687,0.000437,0.034731,0.035948,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.101266,0.000010,0.000010,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.041667,0.027778,0.041667,0.033333,0.027778,0.023810,0.020833,0.018519,0.016667,0.015152,0.021898,0.020134,0.018634,0.017341,0.016216,0.015228,0.014354,0.013575,0.012876,0.012245,0.011673,0.011152,0.010676,0.010239,0.009836,0.009464,0.009146,0.008824,0.008523
30002,model_0,04.22.1750,0.0,0.0,0.000000,0.000000,0.011903,0.001114,0.000372,0.0,0.0,0.0,0.002183,0.002182,0.000000,0.000073,0.113368,0.086928,0.000000,0.000000,0.0,0.000000,0.0,0.003763,0.001510,0.000698,0.000000,0.168776,0.000000,0.000000,0.0,0.0,0.006113,0.000000,0.087432,0.000000,0.000000,0.166667,0.166667,0.000000,0.014508,0.066667,0.000745,0.060606,0.0,0.083333,0.041667,0.055556,0.041667,0.033333,0.083333,0.071429,0.062500,0.064815,0.058333,0.060606,0.058394,0.060403,0.055901,0.052023,0.048649,0.045685,0.047847,0.045249,0.042918,0.040816,0.038911,0.037175,0.035587,0.034130,0.032787,0.031546,0.030488,0.029412,0.028409
30003,model_0,04.22.1750,0.0,0.0,0.000000,0.000000,0.016122,0.001114,0.000744,0.0,0.0,0.0,0.000546,0.000682,0.000172,0.000510,0.005242,0.007843,0.000000,0.024390,0.0,0.043478,0.0,0.000188,0.000075,0.000000,0.000000,0.021097,0.000021,0.000021,0.0,0.0,0.003561,0.000000,0.021858,0.001295,0.003623,0.166667,0.166667,0.000000,0.000000,0.023599,0.005957,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010811,0.015228,0.014354,0.013575,0.012876,0.012245,0.011673,0.011152,0.014235,0.017065,0.016393,0.015773,0.015244,0.014706,0.014205
30004,model_1,04.16.3553,0.0,0.0,0.000000,0.000010,0.000904,0.000557,0.000372,0.0,0.0,0.0,0.001364,0.001636,0.000172,0.001310,0.052425,0.043791,0.013043,0.000000,0.0,0.000000,0.0,0.000627,0.000302,0.000000,0.000000,0.059072,0.000010,0.000010,0.0,0.0,0.008824,0.026965,0.060109,0.000486,0.001208,0.166667,0.166667,0.004274,0.000000,0.028319,0.067014,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004082,0.003891,0.003717,0.003559,0.003413,0.006557,0.006309,0.006098,0.005882,0.005682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,model_1,04.16.3553,0.0,0.0,0.000000,0.000010,0.001808,0.002784,0.001861,0.0,0.0,0.0,0.002183,0.002591,0.000515,0.000655,0.080603,

In [27]:
### fwver 교체

temp1 = train_err[['user_id', 'fwver']]
temp2 = train_prob.copy()
temp3 = pd.merge(temp1, temp2, how = 'inner')
temp3 = temp3.drop('user_id', axis = 1)

fwcnt = pd.DataFrame(temp3.fwver.value_counts())
fwercnt = pd.DataFrame(temp1.fwver.value_counts())

fw = pd.concat([fwercnt, fwcnt], axis = 1, join = 'outer').fillna(0)
fw.columns = ['total', 'reported']
fw['prob'] = fw.reported/fw.total
fw = fw.reset_index().rename(columns={"index": "fwver"})


for i in range(len(trainset)):
  if trainset.iloc[i, 1] in fw.fwver.values:
    trainset.iloc[i, 1] = fw[fw.fwver == trainset.iloc[i, 1]].prob.values
  else:
    trainset.iloc[i, 1] = 0

for i in range(len(testset)):
  if testset.iloc[i, 1] in fw.fwver.values:
    testset.iloc[i, 1] = fw[fw.fwver == testset.iloc[i, 1]].prob.values
  else:
    testset.iloc[i, 1] = 0


In [30]:
trainset = pd.concat([trainset, train_err_code_frq, train_qui_frq], axis = 1)
testset = pd.concat([testset, test_err_code_frq, test_qui_frq], axis = 1)

In [31]:
trainset2 = trainset.reset_index()
testset2 = testset.reset_index()

In [32]:
probs = list(set(train_prob.user_id))

In [33]:
trainset2['TARGET'] = trainset2['index'].apply(lambda x: 1 if x in probs else 0)
trainset2.set_index('index', inplace=True)

In [34]:
X = trainset2.iloc[:,:-1]
y = pd.DataFrame(trainset2.iloc[:,-1])
X

,model_nm,fwver,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30,Most_Err_Code,Second_Err_Code,q1_1,q1_2,q2_1,q2_2,q5_1,q5_2,q6_1,q6_2,q7_1,q7_2,q8_1,q8_2,q9_1,q9_2,q10_1,q10_2,q11_1,q11_2,q12_1,q12_2
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,model_3,0.320613,0.0,0.0,0.001098,0.000533,0.000000,0.000557,0.000372,0.0,0.0,0.000143,0.002047,0.002182,0.000172,0.000728,0.038663,0.039869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135021,0.000010,0.000010,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002941,0.005682,1,0,0.0,None,0.0,None,0.0,4.0,0.0,None,0.0,None,0.0,None,0.0,None,7.0,4.0,0.0,None,0.0,None
10001,model_2,0.463821,0.0,0.0,0.000000,0.000000,0.007986,0.000557,0.000372,0.0,0.0,0.000000,0.001364,0.001500,0.000172,0.001092,0.098952,0.083660,0.000000,0.097561,0.043478,0.130435,0.045455,0.047416,0.028345,0.001745,0.012821,0.092827,0.000000,0.000000,0.0,0.00,0.006644,0.0,0.054645,0.001457,0.0,0.166667,0.166667,0.000000,0.0,0.066667,0.041698,0.030303,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,standby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,model_3,0.320613,0.0,0.0,0.000275,0.000676,0.000151,0.001114,0.000372,0.0,0.0,0.000020,0.001774,0.001909,0.000172,0.000291,0.034076,0.033987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000698,0.000000,0.105485,0.000021,0.000021,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.020833,0.016667,0.027778,0.02381,0.020833,0.018519,0.016667,0.022727,0.021898,0.020134,0.018634,0.023121,0.021622,0.020305,0.023923,0.027149,0.025751,0.024490,0.023346,0.022305,0.021352,0.020478,0.019672,0.018927,0.018293,0.017647,0.022727,1,0,0.0,-1.0,0.0,-1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,None,0.0,1.0,3.0,5.0,0.0,-1.0,0.0,None
10003,model_2,0.463821,0.0,0.0,0.000000,0.000000,0.000301,0.000557,0.000372,0.0,0.0,0.000000,0.001228,0.001227,0.000000,0.000000,0.034076,0.019608,0.002174,0.000000,0.000000,0.000000,0.000000,0.001756,0.000717,0.000000,0.000000,0.248945,0.000000,0.000000,0.0,0.00,0.001728,0.0,0.043716,0.000000,0.0,0.166667,0.166667,0.008547,0.0,0.010029,0.000745,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10004,model_0,0.536741,0.0,0.0,0.000000,0.000005,0.000000,0.001670,0.001489,0.0,0.0,0.000000,0.002183,0.002591,0.000515,0.000364,0.093709,0.059477,0.000000,0.000000,0.000000,0.000000,0.000000,0.008781,0.004491,0.000000,0.000000,0.139241,0.000000,0.000000,0.0,0.00,0.004678,0.0,0.087432,0.000000,0.0,0.166667,0.166667,0.000000,0.0,0.002360,0.000000,0.060606,0.0,0.0,0.041667,0.027778,0.041667,0.033333,0.027778,0.02381,0.020833,0.018519,0.016667,0.015152,0.014599,0.013423,0.012422,0.011561,0.010811,0.0101

In [35]:
val = list(trainset2.columns)

In [36]:
len(val)

98

In [37]:
nots = ['q7_1', 'q7_2', 'q8_1', 'q8_2', 'q9_1', 'q9_2', 'q12_1', 'q12_2']

val2 = [x for x in val if x not in nots]

In [38]:
trainset3 = trainset2[val2]

In [39]:
trainset3

,model_nm,fwver,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30,Most_Err_Code,Second_Err_Code,q1_1,q1_2,q2_1,q2_2,q5_1,q5_2,q6_1,q6_2,q10_1,q10_2,q11_1,q11_2,TARGET
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,model_3,0.320613,0.0,0.0,0.001098,0.000533,0.000000,0.000557,0.000372,0.0,0.0,0.000143,0.002047,0.002182,0.000172,0.000728,0.038663,0.039869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135021,0.000010,0.000010,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002941,0.005682,1,0,0.0,None,0.0,None,0.0,4.0,0.0,None,7.0,4.0,0.0,None,0
10001,model_2,0.463821,0.0,0.0,0.000000,0.000000,0.007986,0.000557,0.000372,0.0,0.0,0.000000,0.001364,0.001500,0.000172,0.001092,0.098952,0.083660,0.000000,0.097561,0.043478,0.130435,0.045455,0.047416,0.028345,0.001745,0.012821,0.092827,0.000000,0.000000,0.0,0.00,0.006644,0.0,0.054645,0.001457,0.0,0.166667,0.166667,0.000000,0.0,0.066667,0.041698,0.030303,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,standby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
10002,model_3,0.320613,0.0,0.0,0.000275,0.000676,0.000151,0.001114,0.000372,0.0,0.0,0.000020,0.001774,0.001909,0.000172,0.000291,0.034076,0.033987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000698,0.000000,0.105485,0.000021,0.000021,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.020833,0.016667,0.027778,0.02381,0.020833,0.018519,0.016667,0.022727,0.021898,0.020134,0.018634,0.023121,0.021622,0.020305,0.023923,0.027149,0.025751,0.024490,0.023346,0.022305,0.021352,0.020478,0.019672,0.018927,0.018293,0.017647,0.022727,1,0,0.0,-1.0,0.0,-1.0,0.0,2.0,0.0,1.0,3.0,5.0,0.0,-1.0,0
10003,model_2,0.463821,0.0,0.0,0.000000,0.000000,0.000301,0.000557,0.000372,0.0,0.0,0.000000,0.001228,0.001227,0.000000,0.000000,0.034076,0.019608,0.002174,0.000000,0.000000,0.000000,0.000000,0.001756,0.000717,0.000000,0.000000,0.248945,0.000000,0.000000,0.0,0.00,0.001728,0.0,0.043716,0.000000,0.0,0.166667,0.166667,0.008547,0.0,0.010029,0.000745,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
10004,model_0,0.536741,0.0,0.0,0.000000,0.000005,0.000000,0.001670,0.001489,0.0,0.0,0.000000,0.002183,0.002591,0.000515,0.000364,0.093709,0.059477,0.000000,0.000000,0.000000,0.000000,0.000000,0.008781,0.004491,0.000000,0.000000,0.139241,0.000000,0.000000,0.0,0.00,0.004678,0.0,0.087432,0.000000,0.0,0.166667,0.166667,0.000000,0.0,0.002360,0.000000,0.060606,0.0,0.0,0.041667,0.027778,0.041667,0.033333,0.027778,0.02381,0.020833,0.018519,0.016667,0.015152,0.014599,0.013423,0.012422,0.011561,0.010811,0.010152,0.009569,0.009050,0.008584,0.008163,0.007782,0.007435,0.007117,0.006826,0.006557,0.006309,0.006098,0.005882,0.005682,1,connection timeout,0.0,-1.0,0.0,-1.0,0.0,-1.0,

In [41]:
trainset3_1 = trainset3.dropna()
trainset3_2 = trainset3.fillna('None')
trainset3_2.drop('model_nm', axis = 1, inplace = True)
trainset3_2

,fwver,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30,Most_Err_Code,Second_Err_Code,q1_1,q1_2,q2_1,q2_2,q5_1,q5_2,q6_1,q6_2,q10_1,q10_2,q11_1,q11_2,TARGET
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,0.320613,0.0,0.0,0.001098,0.000533,0.000000,0.000557,0.000372,0.0,0.0,0.000143,0.002047,0.002182,0.000172,0.000728,0.038663,0.039869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135021,0.000010,0.000010,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002941,0.005682,1,0,0.0,None,0.0,None,0.0,4.0,0.0,None,7.0,4.0,0.0,None,0
10001,0.463821,0.0,0.0,0.000000,0.000000,0.007986,0.000557,0.000372,0.0,0.0,0.000000,0.001364,0.001500,0.000172,0.001092,0.098952,0.083660,0.000000,0.097561,0.043478,0.130435,0.045455,0.047416,0.028345,0.001745,0.012821,0.092827,0.000000,0.000000,0.0,0.00,0.006644,0.0,0.054645,0.001457,0.0,0.166667,0.166667,0.000000,0.0,0.066667,0.041698,0.030303,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,standby,None,None,None,None,None,None,None,None,None,None,None,None,1
10002,0.320613,0.0,0.0,0.000275,0.000676,0.000151,0.001114,0.000372,0.0,0.0,0.000020,0.001774,0.001909,0.000172,0.000291,0.034076,0.033987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000698,0.000000,0.105485,0.000021,0.000021,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.020833,0.016667,0.027778,0.02381,0.020833,0.018519,0.016667,0.022727,0.021898,0.020134,0.018634,0.023121,0.021622,0.020305,0.023923,0.027149,0.025751,0.024490,0.023346,0.022305,0.021352,0.020478,0.019672,0.018927,0.018293,0.017647,0.022727,1,0,0.0,-1.0,0.0,-1.0,0.0,2.0,0.0,1.0,3.0,5.0,0.0,-1.0,0
10003,0.463821,0.0,0.0,0.000000,0.000000,0.000301,0.000557,0.000372,0.0,0.0,0.000000,0.001228,0.001227,0.000000,0.000000,0.034076,0.019608,0.002174,0.000000,0.000000,0.000000,0.000000,0.001756,0.000717,0.000000,0.000000,0.248945,0.000000,0.000000,0.0,0.00,0.001728,0.0,0.043716,0.000000,0.0,0.166667,0.166667,0.008547,0.0,0.010029,0.000745,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,None,None,None,None,None,None,None,None,None,None,None,None,0
10004,0.536741,0.0,0.0,0.000000,0.000005,0.000000,0.001670,0.001489,0.0,0.0,0.000000,0.002183,0.002591,0.000515,0.000364,0.093709,0.059477,0.000000,0.000000,0.000000,0.000000,0.000000,0.008781,0.004491,0.000000,0.000000,0.139241,0.000000,0.000000,0.0,0.00,0.004678,0.0,0.087432,0.000000,0.0,0.166667,0.166667,0.000000,0.0,0.002360,0.000000,0.060606,0.0,0.0,0.041667,0.027778,0.041667,0.033333,0.027778,0.02381,0.020833,0.018519,0.016667,0.015152,0.014599,0.013423,0.012422,0.011561,0.010811,0.010152,0.009569,0.009050,0.008584,0.008163,0.007782,0.007435,0.007117,0.006826,0.006557,0.006309,0.006098,0.005882,0.005682,1,connection timeout,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,1.0,3.0,0.0,-1.0,

In [42]:
testset2.set_index('index', inplace = True)

In [43]:
val2.remove('TARGET')

In [44]:
testset2 = testset2[val2]
testset2.drop('model_nm', axis = 1, inplace = True)
testset2

,fwver,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30,Most_Err_Code,Second_Err_Code,q1_1,q1_2,q2_1,q2_2,q5_1,q5_2,q6_1,q6_2,q10_1,q10_2,q11_1,q11_2
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30000,0.552381,0.0,0.0,0.000000,0.000000,0.009342,0.000557,0.000372,0.0,0.0,0.0,0.002183,0.002182,0.000000,0.000218,0.043906,0.024183,0.019565,0.073171,0.0,0.043478,0.0,0.061653,0.049368,0.001745,0.025641,0.135021,0.000000,0.000000,0.0,0.0,0.002206,0.000355,0.087432,0.000000,0.001208,0.500000,0.500000,0.000000,0.002073,0.054277,0.000000,0.060606,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003049,0.002941,0.002841,1,connection timeout,0.0,None,0.0,None,0.0,2.0,0.0,1.0,2.0,None,0.0,None
30001,0.320613,0.0,0.0,0.000412,0.000502,0.000452,0.002227,0.001489,0.0,0.0,0.0,0.001637,0.002182,0.000687,0.000437,0.034731,0.035948,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.101266,0.000010,0.000010,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.041667,0.027778,0.041667,0.033333,0.027778,0.023810,0.020833,0.018519,0.016667,0.015152,0.021898,0.020134,0.018634,0.017341,0.016216,0.015228,0.014354,0.013575,0.012876,0.012245,0.011673,0.011152,0.010676,0.010239,0.009836,0.009464,0.009146,0.008824,0.008523,1,0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,15.0,4.0,0.0,-1.0
30002,0.536741,0.0,0.0,0.000000,0.000000,0.011903,0.001114,0.000372,0.0,0.0,0.0,0.002183,0.002182,0.000000,0.000073,0.113368,0.086928,0.000000,0.000000,0.0,0.000000,0.0,0.003763,0.001510,0.000698,0.000000,0.168776,0.000000,0.000000,0.0,0.0,0.006113,0.000000,0.087432,0.000000,0.000000,0.166667,0.166667,0.000000,0.014508,0.066667,0.000745,0.060606,0.0,0.083333,0.041667,0.055556,0.041667,0.033333,0.083333,0.071429,0.062500,0.064815,0.058333,0.060606,0.058394,0.060403,0.055901,0.052023,0.048649,0.045685,0.047847,0.045249,0.042918,0.040816,0.038911,0.037175,0.035587,0.034130,0.032787,0.031546,0.030488,0.029412,0.028409,1,0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,3.0,15.0,0.0,-1.0
30003,0.536741,0.0,0.0,0.000000,0.000000,0.016122,0.001114,0.000744,0.0,0.0,0.0,0.000546,0.000682,0.000172,0.000510,0.005242,0.007843,0.000000,0.024390,0.0,0.043478,0.0,0.000188,0.000075,0.000000,0.000000,0.021097,0.000021,0.000021,0.0,0.0,0.003561,0.000000,0.021858,0.001295,0.003623,0.166667,0.166667,0.000000,0.000000,0.023599,0.005957,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010811,0.015228,0.014354,0.013575,0.012876,0.012245,0.011673,0.011152,0.014235,0.017065,0.016393,0.015773,0.015244,0.014706,0.014205,1,B-A8002,0.0,None,0.0,None,0.0,1.0,0.0,1.0,1.0,0.0,0.0,None
30004,0.552381,0.0,0.0,0.000000,0.000010,0.000904,0.000557,0.000372,0.0,0.0,0.0,0.001364,0.001636,0.000172,0.001310,0.052425,0.043791,0.013043,0.000000,0.0,0.000000,0.0,0.000627,0.000302,0.000000,0.000000,0.059072,0.000010,0.000010,0.0,0.0,0.008824,0.026965,0.060109,0.000486,0.001208,0.166667,0.166667,0.004274,0.000000,0.028319,0.067014,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004082,0.003891,0.003717,0.003559,0.003413,0.006557,0.006309,0.006098,0.005882,0.005682,1,0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,2.0,0.0,-1.0
...

In [45]:
clf = setup(data = trainset3_2, target = 'TARGET') 

,Description,Value
0,session_id,7328
1,Target,TARGET
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(15000, 89)"
5,Missing Values,False
6,Numeric Features,73
7,Categorical Features,15
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best_5 = compare_models(sort = 'Accuracy', n_select = 3)

IntProgress(value=0, description='Processing: ', max=79)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:37:13
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7841,0.8033,0.4522,0.8036,0.5783,0.4482,0.4818,1.923
rf,Random Forest Classifier,0.7765,0.7974,0.4461,0.7777,0.5665,0.4305,0.4609,1.061
ada,Ada Boost Classifier,0.7730,0.7866,0.4568,0.7543,0.5687,0.4271,0.4520,0.497
et,Extra Trees Classifier,0.7711,0.7797,0.4632,0.7426,0.5703,0.4254,0.4476,1.073
lda,Linear Discriminant Analysis,0.7547,0.7374,0.3522,0.7792,0.4841,0.3524,0.4005,0.176
ridge,Ridge Classifier,0.7507,0.0000,0.3275,0.7891,0.4623,0.3346,0.3896,0.027
svm,SVM - Linear Kernel,0.7440,0.0000,0.3200,0.8087,0.4410,0.3162,0.3820,0.191
lr,Logistic Regression,0.7409,0.7351,0.3258,0.7378,0.4517,0.3140,0.3591,0.778
nb,Naive Bayes,0.7192,0.7133,0.3284,0.6462,0.4321,0.2717,0.3007,0.034
knn,K Neighbors Classifier,0.6838,0.6470,0.3156,0.5295,0.3953,0.1995,0.2118,1.807


In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
# cat = create_model('catboost')
# tuned_cat = tune_model(cat, optimize='AUC')

In [ ]:
# 앙상블로 예측하고 싶으면 model = final_model, 캣부로 해보고 싶으면 model = tuned_cat
model = final_model

predictions = predict_model(model, data = testset2)

In [ ]:
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i+30000] ==1:
    x.append(predictions['Score'][i+30000])
  else:
    x.append(1-predictions['Score'][i+30000])

In [ ]:
df = pd.DataFrame(model.feature_importances_, index = model.feature_names_, columns = ['feature'])

AttributeError: ignored

In [ ]:
df = df.reset_index()

In [ ]:
# df.to_csv("F_impo.csv", index = False)
# !cp F_impo.csv "drive/My Drive/"

In [ ]:
sample_submssion = pd.read_csv('/content/drive/Shareddrives/dacon/404/sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv("None_cat2.csv", index = False)
!cp None_cat2.csv "drive/My Drive/"